In [5]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [ ]:
npi_specialty = []

In [ ]:
def get_primary_tax(row):
    if row['Healthcare Provider Primary Taxonomy Switch_1'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_1']
    elif row['Healthcare Provider Primary Taxonomy Switch_2'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_2']
    elif row['Healthcare Provider Primary Taxonomy Switch_3'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_3']
    elif row['Healthcare Provider Primary Taxonomy Switch_4'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_4']
    elif row['Healthcare Provider Primary Taxonomy Switch_5'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_5']
    elif row['Healthcare Provider Primary Taxonomy Switch_6'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_6']
    elif row['Healthcare Provider Primary Taxonomy Switch_7'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_7']
    elif row['Healthcare Provider Primary Taxonomy Switch_8'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_8']
    elif row['Healthcare Provider Primary Taxonomy Switch_9'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_9']
    elif row['Healthcare Provider Primary Taxonomy Switch_10'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_10']
    elif row['Healthcare Provider Primary Taxonomy Switch_11'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_11']
    elif row['Healthcare Provider Primary Taxonomy Switch_12'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_12']
    elif row['Healthcare Provider Primary Taxonomy Switch_13'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_13']
    elif row['Healthcare Provider Primary Taxonomy Switch_14'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_14']
    elif row['Healthcare Provider Primary Taxonomy Switch_14'] == 'Y':
        tax = row['Healthcare Provider Taxonomy Code_14']
    else:        
        tax = ''
    return tax

In [ ]:
#this produces a lot of warnings if you run it without the low_memory parameter set
for chunk in tqdm(pd.read_csv('../data/npidata_pfile_20050523-20210207.csv', chunksize = 50000, low_memory=False)):
    for index, row in chunk.iterrows():
        npi = row['NPI']
        primary_tax = get_primary_tax(row)
        npi_specialty.append([npi, primary_tax])

In [ ]:
npi_specialty = pd.DataFrame(npi_specialty)
npi_specialty.columns = ['npi','primary_taxonomy']

In [ ]:
tax_desc = pd.read_csv('../data/nucc_taxonomy_201.csv')

In [ ]:
tax_desc = tax_desc[['Code','Classification']]

In [ ]:
npi_specialty = npi_specialty.merge(tax_desc, how='left', left_on='primary_taxonomy', right_on = 'Code')

In [ ]:
npi_specialty.drop(columns = ['Code'], inplace=True)

In [ ]:
npi_specialty.to_csv('../data/npi_specialty.csv',index=False)

In [2]:
#npi_specialty = pd.read_csv('../data/npi_specialty.csv')

In [3]:
npi_specialty.head(10)

,npi,primary_taxonomy,Classification
0,1679576722,207X00000X,Orthopaedic Surgery
1,1588667638,207RC0000X,Internal Medicine
2,1497758544,251G00000X,"Hospice Care, Community Based"
3,1306849450,2085R0202X,Radiology
4,1215930367,207RH0003X,Internal Medicine
5,1023011178,251G00000X,"Hospice Care, Community Based"
6,1932102084,207RC0000X,Internal Medicine
7,1841293990,231H00000X,Audiologist
8,1750384806,207R00000X,Internal Medicine
9,1669475711,208000000X,Pediatrics


In [6]:
db = sqlite3.connect('../data/hop_teaming.sqlite')

In [7]:
for chunk in tqdm(pd.read_csv('../data/npi_specialty.csv', chunksize = 50000)):
    chunk.to_sql('specialty', db, if_exists = 'append', index = False)            # Append the chunk to a calls table

In [8]:
query = '''
select * from specialty limit 10
'''

In [14]:
test = pd.read_sql(query,db)

In [15]:
test

,count(*),count(distinct npi)
0,6714038,6714038


In [12]:
test.shape

(10, 3)

In [13]:
query = '''
select count(*), count(distinct npi) from specialty
'''